In [1]:
spark

In [2]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
from pyspark.sql.types import Row
import tensorflow 
from tensorflow import keras 
#import tensorflow.keras.experimental.WideDeepModel
import numpy as np
import pandas as pd
import os
import argparse
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, concatenate, Embedding, Reshape
from tensorflow.keras.layers import Flatten, concatenate, Lambda, Dropout, BatchNormalization
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2, l1_l2

tensorflow.__version__

INFO:matplotlib.font_manager:Generating new fontManager, this may take some time...
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [5]:

import warnings
warnings.filterwarnings('ignore')
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, window
import pyspark.sql.functions as F

import time
import datetime
from datetime import date, datetime, timedelta
from os import path
import os
from argparse import ArgumentParser
import json
import subprocess
import math
#import httplib
import re
import ast

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
#%matplotlib inline

#from plotly.offline import init_notebook_mode,iplot, plot
#import plotly.graph_objs as go
#import chart_studio.plotly as py
from IPython.display import display, Markdown,HTML
from operator import add
 

sparkSession = SparkSession.builder.appName("tsc_stockprediction").getOrCreate()    # SparkSession should be assigned.
#init_notebook_mode(connected=True)


In [4]:
import pandas as pd
import numpy as np
def rnd_timeserie(min_date, max_date):
    time_index = pd.date_range(min_date, max_date)
    dates = (pd.DataFrame({'ds': pd.to_datetime(time_index.values)},
                          index=range(len(time_index))))
    y = np.random.random_sample(len(dates))*10
    dates['y'] = y
    return dates

#time_index = pd.date_range(min_date, max_date)

In [5]:
#FB.csv contains the stock price 
facebook = spark.read.csv('Data/FB.csv', header=True, sep=',')
#facebook = facebook.rdd.map(lambda x: Row(ds=pd.to_datetime(x['Date']), y=float(x['Close']))).toDF()
facebook.show(10)
facebook = facebook.rdd.map(lambda x: Row(ds=x['Date'], y=float(x['Close']))).toDF()
facebook = pd.DataFrame(facebook.collect())#

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2020-01-27|213.100006|216.059998|212.500000|214.869995|214.869995|13810700|
|2020-01-28|216.139999|218.740005|213.520004|217.789993|217.789993|13413800|
|2020-01-29|221.440002|224.199997|218.449997|223.229996|223.229996|33181300|
|2020-01-30|206.529999|211.429993|204.750000|209.529999|209.529999|48775500|
|2020-01-31|208.429993|208.690002|201.059998|201.910004|201.910004|31359900|
|2020-02-03|203.440002|205.139999|202.500000|204.190002|204.190002|15510500|
|2020-02-04|206.619995|210.600006|205.199997|209.830002|209.830002|19628900|
|2020-02-05|212.509995|212.729996|208.710007|210.110001|210.110001|12538200|
|2020-02-06|210.470001|211.190002|209.339996|210.850006|210.850006|10567500|
|2020-02-07|210.300003|212.820007|209.929993|212.330002|212.330002|12242500|

In [9]:
facebook = spark.read.csv('Data/FB.csv', header=True, sep=',') 
facebook = facebook.rdd.map(lambda x: Row(ds=x['Date'], y=float(x['Close']))).toDF()
facebook = pd.DataFrame(facebook.collect(), columns=['ds', 'y'])#

In [10]:
facebook

,ds,y
0,2020-01-27,214.869995
1,2020-01-28,217.789993
2,2020-01-29,223.229996
3,2020-01-30,209.529999
4,2020-01-31,201.910004
...,...,...
248,2021-01-20,267.480011
249,2021-01-21,272.869995
250,2021-01-22,274.500000
251,2021-01-25,278.010010


In [18]:
#facebook.rename(columns={'close': 'y', 'date': 'ds'},inplace=True)
#facebook['ds'] = pd.to_datetime(facebook['Date'])
#facebook['y']=facebook['Close'].astype(float)

#facebook.withColumn('ds', pd.to_datetime(facebook['Date']))

#facebook['y']=facebook['y'].astype(float)
m = Prophet() #Prophet(changepoint_prior_scale=0.15, daily_seasonality=False)

m.fit(facebook)
future = m.make_future_dataframe(periods=365)
future = m.predict(future)
future

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-27,218.018946,203.238385,231.424439,218.018946,218.018946,-0.612417,-0.612417,-0.612417,-0.612417,-0.612417,-0.612417,0.0,0.0,0.0,217.406529
1,2020-01-28,217.064170,204.848784,231.437043,217.064170,217.064170,1.030176,1.030176,1.030176,1.030176,1.030176,1.030176,0.0,0.0,0.0,218.094346
2,2020-01-29,216.109394,204.119770,231.724952,216.109394,216.109394,1.685259,1.685259,1.685259,1.685259,1.685259,1.685259,0.0,0.0,0.0,217.794653
3,2020-01-30,215.154618,202.551943,229.633562,215.154618,215.154618,0.970368,0.970368,0.970368,0.970368,0.970368,0.970368,0.0,0.0,0.0,216.124986
4,2020-01-31,214.199841,201.811305,228.280915,214.199841,214.199841,0.952360,0.952360,0.952360,0.952360,0.952360,0.952360,0.0,0.0,0.0,215.152202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,2022-01-22,248.657903,27.944995,439.228738,31.525709,444.882899,-2.012873,-2.012873,-2.012873,-2.012873,-2.012873,-2.012873,0.0,0.0,0.0,246.645030
614,2022-01-23,248.602126,30.211113,442.907013,30.431067,445.944190,-2.012873,-2.012873,-2.012873,-2.012873,-2.012873,-2.012873,0.0,0.0,0.0,246.589252
615,2022-01-24,248.546348,26.185010,442.883306,29.460378,446.932902,-0.612417,-0.612417,-0.612417,-0.612417,-0.612417,-0.612417,0.0,0.0,0.0,247.933931
616,2022-01-25,248.490570,28.149373,449.925755,28.668987,447.714502,1.030176,1.030176,1.030176,1.030176,1.030176,1.030176,0.0,0.0,0.0,249.520746


In [16]:


future.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
613,2022-01-22,196.213337,-476.878513,862.340041,-482.720051,853.517040,5.911667,5.911667,5.911667,6.75619,6.75619,6.75619,-0.844523,-0.844523,-0.844523,0.0,0.0,0.0,202.125004
614,2022-01-23,196.040837,-480.817729,870.205088,-486.185591,854.797212,5.911666,5.911666,5.911666,6.75619,6.75619,6.75619,-0.844524,-0.844524,-0.844524,0.0,0.0,0.0,201.952503
615,2022-01-24,195.868336,-486.345829,871.828765,-490.616030,856.786100,6.021368,6.021368,6.021368,6.75619,6.75619,6.75619,-0.734823,-0.734823,-0.734823,0.0,0.0,0.0,201.889704
616,2022-01-25,195.695836,-483.277755,869.606931,-495.043309,860.469843,7.309883,7.309883,7.309883,6.75619,6.75619,6.75619,0.553693,0.553693,0.553693,0.0,0.0,0.0,203.005719
617,2022-01-26,195.523335,-485.877597,879.259685,-499.470588,864.486611,7.872121,7.872121,7.872121,6.75619,6.75619,6.75619,1.115931,1.115931,1.115931,0.0,0.0,0.0,203.395457


In [21]:
time_index = pd.date_range('2018-01-01','2018-12-30')
dates = (pd.DataFrame({'ds': pd.to_datetime(time_index.values)},
                          index=range(len(time_index))))

y = np.random.random_sample(len(dates))*10
dates['y'] = y 
series = [rnd_timeserie('2018-01-01','2018-12-30') for x in range(0,500)]

In [ ]:
model = Prophet(yearly_seasonality=False,daily_seasonality=False)
model.fit(timeserie)
 

forecast = model.make_future_dataframe(periods=90, include_history=False)

In [32]:

start_time = time.time()

from fbprophet import Prophet
def run_prophet(timeserie):
    model = Prophet(yearly_seasonality=False,daily_seasonality=False)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=90, include_history=False)
    forecast = model.predict(forecast)
    return forecast

f = run_prophet(series[0])
f.head


print("--- %s seconds ---" % (time.time() - start_time))

<bound method NDFrame.head of            ds     trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0  2018-12-31  4.960533    1.282104    8.611046     4.960533     4.960533   
1  2019-01-01  4.959909    1.204651    8.299311     4.959909     4.959909   
2  2019-01-02  4.959284    1.208484    8.528293     4.959284     4.959284   
3  2019-01-03  4.958660    2.055180    9.481777     4.958660     4.958660   
4  2019-01-04  4.958036    1.137141    8.444690     4.958034     4.958037   
..        ...       ...         ...         ...          ...          ...   
85 2019-03-26  4.907478    1.105360    8.388285     4.906916     4.908015   
86 2019-03-27  4.906854    1.123860    8.159563     4.906284     4.907399   
87 2019-03-28  4.906229    1.835688    9.336804     4.905651     4.906784   
88 2019-03-29  4.905605    0.795676    8.238709     4.905015     4.906166   
89 2019-03-30  4.904981    1.234057    8.113824     4.904385     4.905553   

    additive_terms  additive_terms_lower  add

In [20]:
rs = model.predict(forecast)

In [21]:
rs

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-12-31,4.983561,0.978739,8.166133,4.983561,4.983561,-0.377805,-0.377805,-0.377805,-0.377805,-0.377805,-0.377805,0.0,0.0,0.0,4.605756
1,2019-01-01,4.982899,1.153102,8.424277,4.982899,4.982899,-0.187597,-0.187597,-0.187597,-0.187597,-0.187597,-0.187597,0.0,0.0,0.0,4.795301
2,2019-01-02,4.982236,1.665712,8.812734,4.982236,4.982236,0.098636,0.098636,0.098636,0.098636,0.098636,0.098636,0.0,0.0,0.0,5.080872
3,2019-01-03,4.981573,0.871386,7.799092,4.981573,4.981573,-0.582043,-0.582043,-0.582043,-0.582043,-0.582043,-0.582043,0.0,0.0,0.0,4.399530
4,2019-01-04,4.980911,1.978279,9.202600,4.980910,4.980911,0.524453,0.524453,0.524453,0.524453,0.524453,0.524453,0.0,0.0,0.0,5.505364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2019-03-26,4.927231,0.940802,8.219766,4.927210,4.927252,-0.187597,-0.187597,-0.187597,-0.187597,-0.187597,-0.187597,0.0,0.0,0.0,4.739634
86,2019-03-27,4.926568,1.626728,8.508689,4.926546,4.926589,0.098636,0.098636,0.098636,0.098636,0.098636,0.098636,0.0,0.0,0.0,5.025205
87,2019-03-28,4.925906,0.789989,7.970557,4.925883,4.925927,-0.582043,-0.582043,-0.582043,-0.582043,-0.582043,-0.582043,0.0,0.0,0.0,4.343863
88,2019-03-29,4.925243,1.778835,9.119585,4.925220,4.925265,0.524453,0.524453,0.524453,0.524453,0.524453,0.524453,0.0,0.0,0.0,5.449696
